In [1]:
import sys

In [2]:
import extract_dataframe as exdf
import clean_tweets_dataframe as cldf

In [3]:
_, global_tweets_data = exdf.read_json('data/global_twitter_data.json')

In [4]:
len(global_tweets_data)

22000

In [5]:
global_tweets = exdf.TweetDfExtractor(global_tweets_data)

In [6]:
df_global = global_tweets.get_tweet_df(save = True, file_name = 'data/global_processed_tweets.csv')

File Successfully Saved.!!!


In [7]:
_, africa_tweets_data = exdf.read_json('data/africa_twitter_data.json')

In [8]:
len(africa_tweets_data)

28608

In [9]:
africa_tweets = exdf.TweetDfExtractor(africa_tweets_data)

In [10]:
df_africa = africa_tweets.get_tweet_df(save = True, file_name = 'data/africa_processed_tweets.csv')

File Successfully Saved.!!!
